In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
from matplotlib import rc

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig_dir = "figures"
os.makedirs(fig_dir, exist_ok=True)

psm_out_dir = os.path.join(fig_dir, "psm_example")
os.makedirs(psm_out_dir, exist_ok=True)

In [ ]:
from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection/"
CSM = CarlaScenesManager(data_dir=data_root, split="train")
idx_scene = 3
CDM = CSM.get_scene_dataset_by_index(idx_scene)

## First figure: agent with tracks outside of FOV

In [ ]:
from avstack.modules.perception.fov_estimator import SlowRayTraceBevLidarFovEstimator

fov_estimator = SlowRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)

# get a lidar point cloud
idx_frame = 60
frames = CDM.get_frames(sensor="lidar-0", agent=0)
pc = CDM.get_lidar(frame=frames[idx_frame], sensor="lidar-0", agent=0)
img_front = CDM.get_image(frame=frames[idx_frame], sensor="camera-0", agent=0)
img_back = CDM.get_image(frame=frames[idx_frame], sensor="camera-1", agent=0)

In [ ]:
from avapi.visualize.snapshot import show_image, show_lidar_bev_with_boxes
from PIL import Image
import cv2

###############################
# SHOW THE AGENT FIELD OF VIEW
###############################
objs_view = CDM.get_objects_global(frame=frames[idx_frame], ignore_static_agents=True)

# filter the PC for visualization
pc.data.x = pc.data.x[pc.data.x[:, 0] > -6, :]
pc.data.x = pc.data.x[pc.data.x[:, 1] < 20, :]

# get the agent's field of view model
fov_color = "#C79FEF"
fov_out = fov_estimator(pc)

# get the agent's objects viewed - move the pink one just for visualization
agent = CDM.get_agent(frame=frames[idx_frame], agent=0)
objs_downselect = [obj for obj in objs_view if 4 < obj.distance(agent.position)]
dx = -4
dy = 10
objs_downselect[0].position.x[0] += dx
objs_downselect[0].box.position.x[0] += dx
objs_downselect[0].position.x[1] += dy
objs_downselect[0].box.position.x[1] += dy
box_colors = ["#ef33bd", "#025a59", "#ff2929", "#a2a369"]
# order is: pink, green, red, yellow

# show the image
pc_bev = pc.project_to_2d_bev()
point_color = "black"
img_fov_model = show_lidar_bev_with_boxes(
    pc=pc,
    boxes=objs_downselect,
    background_color="white",
    colormethod="black",
    box_colors=box_colors,
    box_filled=False,
    box_thickness=4,
    fov=fov_out,
    fov_color=fov_color,
    fov_filled_alpha=0.5,
    bev_size=(400, 600),
    inline=True,
    flipx=False,
    flipxy=False,
    show=False,
    return_image=True,
)
# add tracked circles
img_fov_out = img_fov_model.copy()
dx = 0
dy = -50
# green
cv2.circle(
    img_fov_out, center=(100 + dx, 210 + dy), color=(0, 255, 0), radius=40, thickness=3
)
# red
cv2.circle(
    img_fov_out, center=(290 + dx, 155 + dy), color=(255, 0, 0), radius=40, thickness=3
)
# yellow
cv2.circle(
    img_fov_out, center=(448 + dx, 210 + dy), color=(255, 0, 0), radius=40, thickness=3
)
# pink
cv2.circle(
    img_fov_out, center=(232 + dx, 270 + dy), color=(0, 255, 0), radius=40, thickness=3
)

# show the image
show_image(img_fov_out, inline=True, grayscale=False)

im = Image.fromarray(img_fov_out)
im.save(os.path.join(psm_out_dir, "agent_0_fov_with_tracks.pdf"))
im.save(os.path.join(psm_out_dir, "agent_0_fov_with_tracks.png"))

In [ ]:
from avtrust import TrustArray, TrustBetaDistribution

###############################
# SHOW THE CORRESPONDING TRACK TRUSTS
###############################

# trust for the agents
colors_agents = ["#C79FEF", "#a87f4f", "#f3ef2d"]
alphas_agents = [12, 10, 2]
betas_agents = [3, 10, 8]
trust_agents = TrustArray(
    timestamp=0.0,
    trusts=[
        TrustBetaDistribution(
            timestamp=0,
            identifier=i,
            alpha=alpha,
            beta=beta,
        )
        for i, (alpha, beta) in enumerate(zip(alphas_agents, betas_agents))
    ],
)

# trust for the correct assignments
# order is: pink, green, red, yellow
colors_tracks = box_colors
alphas_trks = [10, 20, 7, 1]
betas_trks = [10, 3, 3, 8]
trust_tracks = TrustArray(
    timestamp=0.0,
    trusts=[
        TrustBetaDistribution(
            timestamp=0,
            identifier=i,
            alpha=alpha,
            beta=beta,
        )
        for i, (alpha, beta) in enumerate(zip(alphas_trks, betas_trks))
    ],
)

# plot the trusts
x_trust = np.linspace(0, 1.0, num=200)
fig, axs = plt.subplots(2, 1, figsize=(6, 6))

# track trust
plt.rcParams["font.family"] = "serif"
for ax, label, trusts, colors in zip(
    axs,
    ["Track", "Agent"],
    [trust_tracks, trust_agents],
    [colors_tracks, colors_agents],
):
    for (ID, trust), color in zip(trusts.items(), colors):
        ax.plot(
            x_trust, trust.pdf(x_trust), linewidth=4, color=color, label=f"{label} {ID}"
        )
    # ax.set_title(f"{label} Trust")
    ax.set_xlabel(f"{label} Trust Value", family="serif")
    ax.set_xlim([0, 1])
    ax.set_yticklabels([])
    ax.legend()
    ax.grid()
plt.tight_layout()
plt.savefig(os.path.join(psm_out_dir, "agent_track_trusts.pdf"))
plt.savefig(os.path.join(psm_out_dir, "agent_track_trusts.png"))
plt.show()